# Experiment Pipeline Notebook

This notebook is the main control panel for running a complete experiment from start to finish. It generates test data, gets responses from a model, evaluates those responses, and exports the results for analysis in R.



### How to Use:

The cells are designed to be run from top to bottom. Several cells expect user-set paths and labels.

The cells consist of the following:

1.  Environment Setup and Imports
2.  (Optional) Full Reset
3.  Generate Synthetic Task Data
4.  Main Translation Task
5.  Data Labelling
6.  Ingest Session Logs into Database
7.  Evaluation and LLM Judging
8.  Final Results and Exporting



### Dependencies

Dependencies can be installed with the provided "requirements.txt", or via the following:
```bash
pip install pandas apted google-genai anthropic openai ipykernel
```


### API Keys

This notebook expects API keys to work with language models. The following environment variables must contain valid API keys, depending on the models used:

* "GEMINI_API_KEY": Required for synthetic data generation or prompting Gemini models
* "OPENAI_API_KEY": Required if prompting OpenAI models
* "ANTHROPIC_API_KEY": Required if prompting Anthropic models

You can temporarily and easily set these environment variables from this notebook in the first code cell. Google offers free API keys at https://aistudio.google.com/app/apikey

### 1. Environment Setup and Imports

This cell imports all necessary functions from the project's codebase and defines the file paths required for later.

In [1]:
# Used by notebook
import pandas as pd
import os
import shutil
import hashlib

# Code from scripts
from data_utils import create_connection, init_database, get_record_count, get_record_status, ingest_log_files
from data_generation import generate_spell_tasks_procedural, generate_spell_tasks_ex_nihilo, generate_automata_scripting_tasks
from nl_to_dsl import run_prompting_session
from evaluation import process_unjudged_instances

# Define paths
db_file = "evaluation.db"
input_data_path = "synthetic_tasks"
session_log_path = "session_logs"
prompt_file_path = "game_prompts"

# Ensure paths exist
os.makedirs(input_data_path, exist_ok=True)
os.makedirs(session_log_path, exist_ok=True)
os.makedirs(prompt_file_path, exist_ok=True)

# Set temporary environment variables
#os.environ["GEMINI_API_KEY"] = "paste key here"
#os.environ["OPENAI_API_KEY"] = "paste key here"
#os.environ["ANTHROPIC_API_KEY"] = "paste key here"

# Check that environment variables can be accessed
for var in ["GEMINI_API_KEY", "OPENAI_API_KEY", "ANTHROPIC_API_KEY"]:
    if os.getenv(var):
        print(f"{var} was found!")
    else:
        print(f"{var} could not be located.")


GEMINI_API_KEY was found!
OPENAI_API_KEY was found!
ANTHROPIC_API_KEY was found!


### 2. (Optional) Full Reset

This cell resets the working database and clears the synthetic data folders.

In [3]:
conn = create_connection(db_file)

# Reset database
if conn:
    init_database(conn)
    conn.close()

# Clear synthetic inputs
if os.path.exists(input_data_path):
    shutil.rmtree(input_data_path)
    os.makedirs(input_data_path)
    print(f"Cleared directory: {input_data_path}")

# Clear generated logs
if os.path.exists(session_log_path):
    shutil.rmtree(session_log_path)
    os.makedirs(session_log_path)
    print(f"Cleared directory: {session_log_path}")

Connected to database at evaluation.db
Database ready!
Cleared directory: synthetic_tasks
Cleared directory: session_logs


### 3. Generate Synthetic Task Data

This cell handles synthetic input data generation. First, set the desired task type to either "spellScripting" or "automataScripting". Then, uncomment the necessary generator function to create the data. These functions assume a Gemini API Key environment variable.

In [4]:
# 1 - Specify the task type to generate data for
task_type = "spellScripting"
task_file = os.path.join(input_data_path, f"{task_type}_tasks.json")

# 2 - Uncomment the generation method to use

#generate_automata_scripting_tasks(num_systems=1, num_tasks_per_system=3, output_file=task_file, model_name='gemini-2.5-flash')
generate_spell_tasks_procedural(num_tasks=1, output_file=task_file, model_name='gemini-2.5-flash')
#generate_spell_tasks_ex_nihilo(num_tasks=2, max_sentences=2, output_file=task_file, model_name='gemini-2.5-flash')


Generating 1 procedural spells...
 - Generating base spell 1/1...
generating description for: technical_summary (code: 1)
Success: Received response from model.
generating description for: technical_detailed (code: 2)
Success: Received response from model.
generating description for: narrative_summary (code: 3)
Success: Received response from model.
generating description for: narrative_detailed (code: 4)
Success: Received response from model.

Saved 4 spellScripting tasks to synthetic_tasks\spellScripting_tasks.json


### 4. Main Translation Task

This cell reads a prompt template from a file, and then runs the main NL-to-DSL pipeline using your chosen LLM. The prompting strategy is determined by the prompt file, while the model to use is typed directly. This function expects an environment variable with an API key for the chosen model.

In [5]:
# 1 - Point to the appropriate prompt file here
prompt_file = os.path.join(prompt_file_path, "spellScriptingFewShotPlanning.txt")

with open(prompt_file, 'r') as f:
    prompt_text = f.read()
    print(f"Prompt file read from: {prompt_file}")

# 2 - Execute the session with the chosen model
run_prompting_session(
    task_type=task_type,
    model_name='gemini-2.5-flash',
    prompt_preamble=prompt_text,
    input_json_file=task_file,
    output_log_dir=session_log_path,
    delay=0
)

# The following model names were used for analysis:
# 'gemini-2.5-flash' 'gpt-4.1-mini' 'claude-sonnet-4-20250514' 'gemma-3-4b-it'

Prompt file read from: game_prompts\spellScriptingFewShotPlanning.txt
Starting new prompting session...
Task type: spellScripting
Model: gemini-2.5-flash
Input file: synthetic_tasks\spellScripting_tasks.json

Loaded 4 tasks from synthetic_tasks\spellScripting_tasks.json.
Will write results to: session_logs\LatentSpaceLog-gemini-2.5-flash-2025-08-13-10-17-31.txt

Processing task 1/4...
Success: Received response from model.
Processing task 2/4...
Success: Received response from model.
Processing task 3/4...
Success: Received response from model.
Processing task 4/4...
Success: Received response from model.
All 4 tasks have been processed and logged to session_logs\LatentSpaceLog-gemini-2.5-flash-2025-08-13-10-17-31.txt.


### 5. Data Labelling

The code expects session logs to be organised in a folder structure with group labels. Use the following cell to name them according to the prompt file used. For example, it is currently set to "fewshot-planning", but could also be "oneshot", or "zeroshot-planning".

In [6]:
# Add a descriptive label to the model group name for this session
# Set to an empty string "" to use only the model name
group_label = "fewshot-planning"

files_to_move = [f for f in os.listdir(session_log_path) if os.path.isfile(os.path.join(session_log_path, f))]

for filename in files_to_move:
    if filename.startswith("LatentSpaceLog-") and filename.endswith(".txt"):
        # Parse filename like: LatentSpaceLog-{model_name}-{timestamp}.txt
        base_name = filename.replace('LatentSpaceLog-', '').replace('.txt', '')
        parts = base_name.split('-')
        session_timestamp = '-'.join(parts[-6:])
        base_model_name = '-'.join(parts[:-6])
        
        # Append the label to the model name to create the final group name.
        final_group_name = f"{base_model_name}-{group_label}" if group_label else base_model_name

        # Create the necessary subdirectory for the final group name
        group_dir = os.path.join(session_log_path, final_group_name)
        os.makedirs(group_dir, exist_ok=True)

        # Move the file into the subdirectory
        old_path = os.path.join(session_log_path, filename)
        new_filename = f"Session-{session_timestamp}.txt"
        new_path = os.path.join(group_dir, new_filename)
        shutil.move(old_path, new_path)
        print(f"Moved {filename} to {final_group_name}/{new_filename}")

Moved LatentSpaceLog-gemini-2.5-flash-2025-08-13-10-17-31.txt to gemini-2.5-flash-fewshot-planning/Session-2025-08-13-10-17-31.txt


### 6. Ingest Session Logs into Database

This step crawls any folders created above and loads them into the database. New records are assigned a "pending" status by default.

In [7]:
ingest_log_files(db_file=db_file, master_folder=session_log_path)

conn = create_connection(db_file)
if conn:
    get_record_count(conn)
    get_record_status(conn)
    conn.close()

Connected to database at evaluation.db
4 new records inserted, 0 records ignored as duplicates.
Connected to database at evaluation.db
Total records in database: 4
Database status:
Status: pending, Count: 4


### 7. Evaluation and LLM Judging

This cell queries the database for all records with a pending status, and then performs syntax validation, calculates similarity scores, and calls the LLM judge as necessary.

In [8]:
# The judging function requires a folder containing prompt components
judge_prompt_dir = "judge_prompts"

process_unjudged_instances(db_file=db_file, prompt_folder=judge_prompt_dir, model_name='gemini-2.5-flash', limit=10, do_llm=True)

Starting judging process for up to 10 records...
Connected to database at evaluation.db
Found 4 records to process.
Success: Received response from model.
Procedural code found for row_id 1. Calculating similarity...
Successfully updated record 1 with judging results.
Success: Received response from model.
Procedural code found for row_id 2. Calculating similarity...
Successfully updated record 2 with judging results.
Success: Received response from model.
Procedural code found for row_id 3. Calculating similarity...
Successfully updated record 3 with judging results.
Success: Received response from model.
Procedural code found for row_id 4. Calculating similarity...
Successfully updated record 4 with judging results.
Judging process finished for this batch!


### 8. Final Results and Exporting

This cell prepares the results in a DataFrame and exports the chosen columns to a CSV file for analysis in R.

In [9]:
# Read final database to a dataframe
conn = create_connection(db_file)
if conn:
    get_record_status(conn)
    
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', 80)
    
    df_final = pd.read_sql_query("SELECT * FROM responses WHERE status = 'judged'", conn)
    display(df_final.head(5))
    conn.close()

# Define all columns to be exported
columns_to_export = [
    'problem_hash',
    'input',
    'model_name',
    'shot_strategy',
    'planning',
    'procedural',
    'procedural_code',
    'judge_rationales_json'
]

# Find all columns starting with score_ or param_
score_cols = [col for col in df_final.columns if col.startswith('score_')]
param_cols = [col for col in df_final.columns if col.startswith('param_')]

# Combine all the column lists
columns_to_export.extend(score_cols)
columns_to_export.extend(param_cols)
columns_to_export = list(dict.fromkeys(columns_to_export))

# Filter the list to only include columns that actually exist
final_columns = [col for col in columns_to_export if col in df_final.columns]
df_for_export = df_final[final_columns].copy()

# For procedurally-generated inputs, the code can be hashed to make it CSV-compatible,
# and the description conditions need to be unpacked
if 'procedural_code' in df_for_export.columns:
    # Create a new column with a SHA256 hash
    df_for_export['procedural_code_hash'] = df_for_export['procedural_code'].apply(
        lambda x: hashlib.sha256(str(x).encode('utf-8')).hexdigest()
    )

    # Remove the original column
    df_for_export.drop(columns=['procedural_code'], inplace=True)
    print("Note: 'procedural_code' was replaced with 'procedural_code_hash'.")

    if 'param_max_sentences_for_desc' in df_for_export.columns:
        # Define the mapping from the integer codes back to the factor levels
        # 1 - Technical, Summary
        # 2 - Technical, Detailed
        # 3 - Narrative, Summary
        # 4 - Narrative, Detailed
        style_mapper = {1: 'technical', 2: 'technical', 3: 'narrative', 4: 'narrative'}
        detail_mapper = {1: 'summary', 2: 'detailed', 3: 'summary', 4: 'detailed'}

        # Create the two new columns
        df_for_export['description_style'] = df_for_export['param_max_sentences_for_desc'].map(style_mapper)
        df_for_export['description_detail'] = df_for_export['param_max_sentences_for_desc'].map(detail_mapper)

        # Remove the original encoded column
        df_for_export.drop(columns=['param_max_sentences_for_desc'], inplace=True)
        print("Note: Decoded 'param_max_sentences_for_desc' into 'description_style' and 'description_detail'.")


# Export
output_filename = 'example_results.csv'
df_for_export.to_csv(output_filename, index=False)

print(f"Successfully exported {len(df_for_export.columns)} columns to {output_filename}")

Connected to database at evaluation.db
Database status:
Status: judged, Count: 4


,row_id,problem_hash,session_name,task_key,input,context,procedural,procedural_code,model_response,status,judge_rationales_json,judged_at,ingested_at,model_name,shot_strategy,planning,score_creative_alignment,score_instructional_precision,score_emergence,score_structural_coherence,score_programmatic_validation,score_tree_edit_distance,score_jaccard_similarity,param_nesting_complexity,param_component_complexity,param_max_sentences_for_desc
0,1,382ccd4134367a392c948c2ab1048f16297bdce579112ea569bb4d4f230fcf61,Session-2025-08-13-10-17-31,spellScripting-2025-08-13-10-17-51,"Launches a controllable projectile that periodically teleports you, heals yo...","[""death"", ""metal"", ""sand"", ""spirit"", ""dream"", ""nether"", ""sound"", ""force"", ""m...",1,"{""friendlyName"": ""Procedural Spell"", ""components"": [{""componentType"": ""proje...","{\n ""planning"": ""The core of the spell is a 'controllable projectile,' whic...",judged,"{""creative_alignment"": ""The spell perfectly captures all aspects of the user...",2025-08-13T10:20:12.346987,2025-08-13T10:19:25.227899,gemini-2.5-flash,fewshot,1,5,5,4,3,Passed,15,0.4762,4,3,1
1,2,ae3c52341c24d99397bca76198856f9a4630726000001543f986ec5502b40222,Session-2025-08-13-10-17-31,spellScripting-2025-08-13-10-18-28,## Spell Mechanics: Procedural Spell\n\nThis document outlines the step-by-s...,"[""death"", ""metal"", ""sand"", ""spirit"", ""dream"", ""nether"", ""sound"", ""force"", ""m...",1,"{""friendlyName"": ""Procedural Spell"", ""components"": [{""componentType"": ""proje...","{\n ""planning"": ""The core spell is a highly-bouncing, controllable projecti...",judged,"{""creative_alignment"": ""The spell captures the core concept of a multi-stage...",2025-08-13T10:20:35.566089,2025-08-13T10:19:25.228565,gemini-2.5-flash,fewshot,1,3,2,2,3,Passed,9,0.7895,4,3,2
2,3,fe09a551936f4e585f69167b51fd6799260cbd6be3a14fb1fe8afcffb83a3b66,Session-2025-08-13-10-17-31,spellScripting-2025-08-13-10-18-44,"A verdant orb of eroding death, guided by will, allows the caster to dance t...","[""death"", ""metal"", ""sand"", ""spirit"", ""dream"", ""nether"", ""sound"", ""force"", ""m...",1,"{""friendlyName"": ""Procedural Spell"", ""components"": [{""componentType"": ""proje...","{\n ""planning"": ""The core of the spell is a 'verdant orb guided by will,' w...",judged,"{""creative_alignment"": ""The spell effectively captures the core concept of a...",2025-08-13T10:20:56.566824,2025-08-13T10:19:25.228648,gemini-2.5-flash,fewshot,1,4,4,4,1,Passed,18,0.4286,4,3,3
3,4,574284eb5cc8a43e3ccf772940b2f24943edf125caed8107f0836daeddfaf5d5,Session-2025-08-13-10-17-31,spellScripting-2025-08-13-10-19-08,"From the caster's outstretched hand, a **""Procedural Spell""** begins its jou...","[""death"", ""metal"", ""sand"", ""spirit"", ""dream"", ""nether"", ""sound"", ""force"", ""m...",1,"{""friendlyName"": ""Procedural Spell"", ""components"": [{""componentType"": ""proje...","{\n ""planning"": ""The core spell is a 'colossal, shimmering orb' that is 'sl...",judged,"{""creative_alignment"": ""The spell successfully captures the core concept of ...",2025-08-13T10:21:44.108742,2025-08-13T10:19:25.228704,gemini-2.5-flash,fewshot,1,4,4,4,5,Passed,11,0.8095,4,3,4


Note: 'procedural_code' was replaced with 'procedural_code_hash'.
Note: Decoded 'param_max_sentences_for_desc' into 'description_style' and 'description_detail'.
Successfully exported 19 columns to example_results.csv
